<a href="https://colab.research.google.com/github/aworklab/everydayPython/blob/main/ex_20230311.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘의 목표
* Customer Analysis
* Iterators

In [2]:
import pandas as pd
import numpy as np
import plotnine as gg
from plotnine import ggplot, aes
import matplotlib.pyplot as plt
import seaborn as sns 
from datetime import datetime, date, time, timezone, timedelta
import os
import re
from functools import reduce 
from collections import defaultdict 
from scipy import stats
from scipy.stats import norm

# Customer Analysis

In [3]:
os.listdir()

['.config',
 'match_result_test.csv',
 'bz_data_analyis.csv',
 'mall_customers.csv',
 'KPMG_VI_New_raw_data_update_final.csv',
 'app_wau.csv',
 'sample-sales-data.csv',
 'sales_vs_discount.csv',
 '1_customers_dataset.csv',
 'cookie_cats.csv',
 'test.csv',
 '1_order_payments_dataset.csv',
 'ab_experiment.csv',
 'WA_Fn-UseC_-Marketing-Customer-Value-Analysis.csv',
 'scanner_data.csv',
 'sales_data_with_stores.csv',
 'lead_conversion.csv',
 'KPMG_VI_New_raw_data_update_final.xlsx',
 'sales.csv',
 'ecommerce-data.csv',
 '1_orders_dataset.csv',
 'online_retail.xlsx',
 'groceries.csv',
 'mock_sales_data.csv',
 'online_retail_II.xlsx',
 'WA_Fn-UseC_-Telco-Customer-Churn.csv',
 'sample_data']

In [7]:
df = pd.read_csv('lead_conversion.csv')
df.columns = [re.sub(' ', '_', a).lower() for a in df.columns]
df.head()

,prospect_id,lead_number,lead_origin,lead_source,do_not_email,do_not_call,converted,totalvisits,total_time_spent_on_website,page_views_per_visit,last_activity,country,specialization,how_did_you_hear_about_x_education,what_is_your_current_occupation,what_matters_most_to_you_in_choosing_a_course,search,magazine,newspaper_article,x_education_forums,newspaper,digital_advertisement,through_recommendations,receive_more_updates_about_our_courses,tags,lead_quality,update_me_on_supply_chain_content,get_updates_on_dm_content,lead_profile,city,asymmetrique_activity_index,asymmetrique_profile_index,asymmetrique_activity_score,asymmetrique_profile_score,i_agree_to_pay_the_amount_through_cheque,a_free_copy_of_mastering_the_interview,last_notable_activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,Page Visited on Website,NaN,Select,Select,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Interested in other courses,Low in Relevance,No,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,Email Opened,India,Select,Select,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Ringing,NaN,No,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,Email Opened,India,Business Administration,Select,Student,Better Career Prospects,No,No,No,No,No,No,No,No,Will revert after reading the email,Might be,No,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,Unreachable,India,Media and Advertising,Word Of Mouth,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Ringing,Not Sure,No,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,Converted to Lead,India,Select,Other,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Will revert after reading the email,Might be,No,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified


In [12]:
df.isnull().sum()

prospect_id                                         0
lead_number                                         0
lead_origin                                         0
lead_source                                        36
do_not_email                                        0
do_not_call                                         0
converted                                           0
totalvisits                                       137
total_time_spent_on_website                         0
page_views_per_visit                              137
last_activity                                     103
country                                          2461
specialization                                   1438
how_did_you_hear_about_x_education               2207
what_is_your_current_occupation                  2690
what_matters_most_to_you_in_choosing_a_course    2709
search                                              0
magazine                                            0
newspaper_article           

In [9]:
df.columns

Index(['prospect_id', 'lead_number', 'lead_origin', 'lead_source',
       'do_not_email', 'do_not_call', 'converted', 'totalvisits',
       'total_time_spent_on_website', 'page_views_per_visit', 'last_activity',
       'country', 'specialization', 'how_did_you_hear_about_x_education',
       'what_is_your_current_occupation',
       'what_matters_most_to_you_in_choosing_a_course', 'search', 'magazine',
       'newspaper_article', 'x_education_forums', 'newspaper',
       'digital_advertisement', 'through_recommendations',
       'receive_more_updates_about_our_courses', 'tags', 'lead_quality',
       'update_me_on_supply_chain_content', 'get_updates_on_dm_content',
       'lead_profile', 'city', 'asymmetrique_activity_index',
       'asymmetrique_profile_index', 'asymmetrique_activity_score',
       'asymmetrique_profile_score',
       'i_agree_to_pay_the_amount_through_cheque',
       'a_free_copy_of_mastering_the_interview', 'last_notable_activity'],
      dtype='object')

In [17]:
cnt = df.groupby(['country', 'converted'])['prospect_id'].count().reset_index(name = 'cnt').sort_values(['cnt'], ascending = False)
cnt['ratio'] = cnt.groupby(['country'])['cnt'].apply(lambda x : round(x/sum(x)*100,2))
cnt[cnt['converted'] == 1]

,country,converted,cnt,ratio
20,India,1,2401,36.98
49,United Arab Emirates,1,20,37.74
53,United States,1,18,26.09
39,Singapore,1,11,45.83
51,United Kingdom,1,5,33.33
5,Bahrain,1,4,57.14
37,Saudi Arabia,1,4,19.05
18,Hong Kong,1,4,57.14
31,Oman,1,3,50.00
3,Australia,1,3,23.08


In [18]:
df = df[df['country'] == 'India']

In [26]:
cnt = df.groupby(['city','what_is_your_current_occupation', 'converted'])['prospect_id'].count().reset_index(name = 'cnt').sort_values(['cnt'], ascending = False)
cnt['ratio'] = cnt.groupby(['city', 'what_is_your_current_occupation'])['cnt'].apply(lambda x : round(x/sum(x)*100,2))
cnt = cnt[cnt['converted'] == 1].reindex(columns = ['city', 'what_is_your_current_occupation', 'ratio'])
pd.pivot_table(data = cnt, index = 'city', columns = 'what_is_your_current_occupation', values = ['ratio'])

ratio  ...                     
what_is_your_current_occupation Businessman  ... Working Professional
city                                         ...                     
Mumbai                                  NaN  ...                91.94
Other Cities                         100.00  ...                95.00
Other Cities of Maharashtra             NaN  ...                94.44
Other Metro Cities                      NaN  ...                84.85
Select                                66.67  ...                95.77
Thane & Outskirts                       NaN  ...                85.45
Tier II Cities                       100.00  ...                83.33

[7 rows x 6 columns]

In [36]:
cnt = df.groupby(['city','what_is_your_current_occupation', 'what_matters_most_to_you_in_choosing_a_course'])['prospect_id'].count().reset_index(name = 'cnt').sort_values(['cnt'], ascending = False)
cnt['ratio'] = cnt.groupby(['city', 'what_is_your_current_occupation'])['cnt'].apply(lambda x : round(x/sum(x)*100,2))
cnt = cnt.reindex(columns = ['city', 'what_is_your_current_occupation', 'what_matters_most_to_you_in_choosing_a_course', 'ratio'])
pd.pivot_table(cnt, index = ['city', 'what_is_your_current_occupation'], columns = 'what_matters_most_to_you_in_choosing_a_course', values = 'ratio')

what_matters_most_to_you_in_choosing_a_course                Better Career Prospects  Other
city                        what_is_your_current_occupation                                
Mumbai                      Housewife                                         100.00    NaN
                            Other                                             100.00    NaN
                            Student                                           100.00    NaN
                            Unemployed                                         99.95   0.05
                            Working Professional                              100.00    NaN
Other Cities                Businessman                                       100.00    NaN
                            Other                                             100.00    NaN
                            Student                                           100.00    NaN
                            Unemployed                                        100.00    NaN
                            Working Professional                              100.00    NaN
Other Cities of Maharashtra Student                                           100.00    NaN
                            Unemployed                                        100.00    NaN
                            Working Professional                              100.00    NaN
Other Metro Cities          Other                                             100.00    NaN
                            Student                                           100.00    NaN
                            Unemployed                                        100.00    NaN
                            Working Professional                              100.00    NaN
Select                      Businessman                                       100.00    NaN
                            Other                                             100.00    NaN
                            Student                                           100.00    NaN
                            Unemployed                                        100.00    NaN
                            Working Professional                              100.00    NaN
Thane & Outskirts           Housewife                                         100.00    NaN
                            Student                                           100.00    NaN
                            Unemployed                                        100.00    NaN
                            Working Professional                              100.00    NaN
Tier II Cities              Businessman                                       100.00    NaN
                            Unemployed                                        100.00    NaN
                            Working Professional                              100.00    NaN

In [48]:
print(len(df))
df[['prospect_id', 'lead_number']].value_counts().reset_index()

6492


,prospect_id,lead_number,0
0,000104b9-23e4-4ddc-8caa-8629fe8ad7f4,654525,1
1,aac1f048-f739-4db8-a1bb-aa3165155a2b,584540,1
2,aba06914-9b01-4850-a1e6-d2d8e40a19fa,616460,1
3,ab9c4307-f9db-4f39-b3cb-c4de3477fae7,601116,1
4,ab95807d-974c-4b2b-b1a0-939703474db7,655777,1
...,...,...,...
6487,53744d5a-0483-42c0-80b0-8990a4d2356d,621166,1
6488,536cdc6b-f4c1-449d-bfd8-9ef0ac912dbb,611052,1
6489,53690d88-52f0-4ce5-b6b8-a13570a6db35,610095,1
6490,5363bd79-576c-48ed-83e4-024c81ea00c5,642895,1


In [54]:
cnt = df[['lead_number','search', 'magazine',
       'newspaper_article', 'x_education_forums', 'newspaper',
       'digital_advertisement', 'through_recommendations']]
cnt = pd.melt(cnt, id_vars = 'lead_number', var_name = 'reason', value_name = 'answer')
cnt = cnt[cnt['answer'] == 'Yes']
cnt.groupby(['reason'])['lead_number'].count().reset_index(name = 'cnt')

,reason,cnt
0,digital_advertisement,4
1,newspaper,1
2,newspaper_article,2
3,search,14
4,through_recommendations,7
5,x_education_forums,1


# Iterators

In [55]:
users = [{'mail': 'gregorythomas@gmail.com', 'name': 'Brett Holland', 'sex': 'M', 'age': 73},
 {'mail': 'hintoncynthia@hotmail.com', 'name': 'Madison Martinez', 'sex': 'F', 'age': 29},
 {'mail': 'wwagner@gmail.com', 'name': 'Michael Jenkins', 'sex': 'M', 'age': 51},
 {'mail': 'daniel79@gmail.com', 'name': 'Karen Rodriguez', 'sex': 'F', 'age': 32},
 {'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F', 'age': 42}]
users

[{'mail': 'gregorythomas@gmail.com',
  'name': 'Brett Holland',
  'sex': 'M',
  'age': 73},
 {'mail': 'hintoncynthia@hotmail.com',
  'name': 'Madison Martinez',
  'sex': 'F',
  'age': 29},
 {'mail': 'wwagner@gmail.com',
  'name': 'Michael Jenkins',
  'sex': 'M',
  'age': 51},
 {'mail': 'daniel79@gmail.com',
  'name': 'Karen Rodriguez',
  'sex': 'F',
  'age': 32},
 {'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F', 'age': 42}]

In [58]:
{k : eval(f'reduce(lambda acc, cur : acc + [cur["{k}"]], users, [])') for k in users[0].keys()}

{'mail': ['gregorythomas@gmail.com',
  'hintoncynthia@hotmail.com',
  'wwagner@gmail.com',
  'daniel79@gmail.com',
  'ujackson@gmail.com'],
 'name': ['Brett Holland',
  'Madison Martinez',
  'Michael Jenkins',
  'Karen Rodriguez',
  'Amber Rhodes'],
 'sex': ['M', 'F', 'M', 'F', 'F'],
 'age': [73, 29, 51, 32, 42]}

In [57]:
{k: [user[k] for user in users] for k in users[0].keys()}

{'mail': ['gregorythomas@gmail.com',
  'hintoncynthia@hotmail.com',
  'wwagner@gmail.com',
  'daniel79@gmail.com',
  'ujackson@gmail.com'],
 'name': ['Brett Holland',
  'Madison Martinez',
  'Michael Jenkins',
  'Karen Rodriguez',
  'Amber Rhodes'],
 'sex': ['M', 'F', 'M', 'F', 'F'],
 'age': [73, 29, 51, 32, 42]}